# Nozzle Flow "Hand" Calculations
This calculates the several parameters and values along the flow of the nozzle from my capstone project using the isentropic flow relations. These values are then used to validate CFD and FEA simulations done in ANSYS Fluent. All units are in SI (m, N, K, etc.)

### Calculating the Exit Temperature
This section calculates the temperature at the exit of the nozzle. This will be compared with the CFD values.

In [1]:
# Importing packages
import pandas as pd
import numpy as np
from scipy.optimize import fsolve

# Initializing
inlet_n = 10
outlet_n = 50
p_total = 600*6894.76
T_total = 2888

# Setting gammas along nozzle
gamma_inlet = np.linspace(1.225, 1.227, 10)
gamma_throat = 1.227
gamma_outlet = np.linspace(1.227, 1.261, 50)

# Setting y values along nozzle
inlet_n = 10
curve_n = 3
outlet_n = 50
y_inlet = np.linspace(0, 0.01094, inlet_n) #m
y_inlet_curve = np.linspace(0.01094, 0.01324, curve_n)
y_throat = [(0.01324+0.01480)/2]
y_outlet = np.linspace(0.01480, 0.07128, outlet_n) #m
ys = np.concatenate((y_inlet, y_inlet_curve, y_throat, y_outlet))

# Setting radii along nozzle
r_inlet = np.linspace(0.0326, 0.009088, inlet_n)
r_inlet_curve = np.linspace(0.009088, 0.00762, curve_n)
r_throat = 0.00762
r_outlet = np.linspace(0.00762, 0.0226, outlet_n)
rs = np.concatenate((r_inlet, r_inlet_curve, [r_throat], r_outlet))

# Setting areas along nozzle
A_inlet = np.pi*r_inlet**2 #m^2
A_inlet_curve = np.pi*r_inlet_curve**2
A_throat = np.pi*r_throat**2
A_outlet = np.pi*r_outlet**2
As = np.concatenate((A_inlet, A_inlet_curve, [A_throat], A_outlet))

# Defining Area-Mach Relation
def area_mach(x, *data): #a_r is area ratio, g is gamma
    a_r, g = data
    return ((g+1)/2)**(-(g+1)/(2*(g-1))) * (1+(g-1)/2 * x**2)**((g+1)/(2*(g-1)))/x - a_r

# Finding Mach at each point
M_inlet = []
for i in range(inlet_n):
    M_inlet = np.append(M_inlet, fsolve(area_mach, 0.1, args=(A_inlet[i]/A_throat, gamma_inlet[i])))
M_inlet_curve = []
for i in range(curve_n):
    M_inlet_curve = np.append(M_inlet_curve, fsolve(area_mach, 0.1, args=(A_inlet_curve[i]/A_throat, gamma_throat)))
M_throat = [1]
M_outlet = []
for i in range(outlet_n):
    M_outlet = np.append(M_outlet, fsolve(area_mach, 5, args=(A_outlet[i]/A_throat, gamma_outlet[i])))
Ms = np.concatenate((M_inlet, M_inlet_curve, M_throat, M_outlet))

# Finding T at each point
Ts_inlet = []
for i in range(inlet_n):
    Ts_inlet = np.append(Ts_inlet, T_total*(1 + (gamma_inlet[i]-1)/2*M_inlet[i]**2)**(-1))
Ts_inlet_curve = []
for i in range(curve_n):
    Ts_inlet_curve = np.append(Ts_inlet_curve, T_total*(1 + (gamma_throat-1)/2*M_inlet_curve[i]**2)**(-1))
Ts_throat = [T_total*(1 + (gamma_throat-1)/2*M_throat[0]**2)**(-1)]
Ts_outlet = []
for i in range(outlet_n):
    Ts_outlet = np.append(Ts_outlet, T_total*(1 + (gamma_outlet[i]-1)/2*M_outlet[i]**2)**(-1))
Ts = np.concatenate((Ts_inlet, Ts_inlet_curve, Ts_throat, Ts_outlet))
print("The exit temperature in Kelvin is", round(Ts[-1], 3))

The exit temperature in Kelvin is 1169.259


The temperature of the casing outside of the nozzle is expected to be much lower than this as the above temperature is directly at the exit within the diameter of the nozzle. Since the flow is over-expanded, we can expect that there won't be any flames directly on the aft end of the casing.

### Calculating the Pressures and Net Force Load on Nozzle
This section will first calculate the pressures along the flow of the nozzle. Then, it will estimate the net force load on the nozzle using an integrative process. The vertical net force will be used to estimate the compressive stress felt by the nozzle and compared to the FEA simulation.

In [2]:
# Finding P at each point
ps_inlet = []
for i in range(inlet_n):
    ps_inlet = np.append(ps_inlet, p_total*(1 + (gamma_inlet[i]-1)/2*M_inlet[i]**2)**(-gamma_inlet[i]/(gamma_inlet[i]-1)))
ps_inlet_curve = []
for i in range(curve_n):
    ps_inlet_curve = np.append(ps_inlet_curve, p_total*(1 + (gamma_throat-1)/2*M_inlet_curve[i]**2)**(-gamma_throat/(gamma_throat-1)))
ps_throat = [p_total*(1 + (gamma_throat-1)/2*M_throat[0]**2)**(-gamma_throat/(gamma_throat-1))]
ps_outlet = []
for i in range(outlet_n):
    ps_outlet = np.append(ps_outlet, p_total*(1 + (gamma_outlet[i]-1)/2*M_outlet[i]**2)**(-gamma_outlet[i]/(gamma_outlet[i]-1)))
ps = np.concatenate((ps_inlet, ps_inlet_curve, ps_throat, ps_outlet))

Pressure acts normal to the surface. Therefore, I must calculate the surface area of each section of the converging and diverging section. The curved area will be ignored in this case to simplify the calculations.

In [3]:
# Finding S.A. of each section. Using the truncated cone surface area formula.
conv_angle = 65
div_angle = 15
inlet_fillet_r = 0.00254
SA_inlet = []
for i in range(inlet_n-1):
    # Using trig to find length of section
    length = (y_inlet[i+1] - y_inlet[i])/np.cos(conv_angle*np.pi/180)
    SA_inlet = np.append(SA_inlet, abs(np.pi*(r_inlet[i] + r_inlet[i+1])*length))
torus_R = r_throat + inlet_fillet_r
torus_r = inlet_fillet_r
SA_inlet_curve = [4*np.pi**2*inlet_fillet_r*torus_R*conv_angle/360]
SA_throat = [abs(2*np.pi*r_throat*(0.01324-0.01480))]
SA_outlet = []
for i in range(outlet_n-1):
    # Using trig to find length of section
    length = (y_outlet[i+1] - y_outlet[i])/np.cos(div_angle*np.pi/180)
    SA_outlet = np.append(SA_outlet, abs(np.pi*(r_outlet[i] + r_outlet[i+1])*length))
SAs = np.concatenate((SA_inlet, SA_inlet_curve, SA_throat, SA_outlet))
SAs

array([5.65541417e-04, 5.18329299e-04, 4.71117181e-04, 4.23905063e-04,
       3.76692945e-04, 3.29480826e-04, 2.82268708e-04, 2.35056590e-04,
       1.87844472e-04, 1.83949248e-04, 7.46894804e-05, 5.82794419e-05,
       6.05716309e-05, 6.28638200e-05, 6.51560090e-05, 6.74481981e-05,
       6.97403871e-05, 7.20325761e-05, 7.43247652e-05, 7.66169542e-05,
       7.89091433e-05, 8.12013323e-05, 8.34935213e-05, 8.57857104e-05,
       8.80778994e-05, 9.03700885e-05, 9.26622775e-05, 9.49544665e-05,
       9.72466556e-05, 9.95388446e-05, 1.01831034e-04, 1.04123223e-04,
       1.06415412e-04, 1.08707601e-04, 1.10999790e-04, 1.13291979e-04,
       1.15584168e-04, 1.17876357e-04, 1.20168546e-04, 1.22460735e-04,
       1.24752924e-04, 1.27045113e-04, 1.29337302e-04, 1.31629491e-04,
       1.33921680e-04, 1.36213869e-04, 1.38506058e-04, 1.40798247e-04,
       1.43090436e-04, 1.45382625e-04, 1.47674814e-04, 1.49967003e-04,
       1.52259193e-04, 1.54551382e-04, 1.56843571e-04, 1.59135760e-04,
      

Now, the average pressures at each section will be calculated. For the inlet curve, I'm doing the average of all the points in that section. I'm essentially doing the midpoint formula to find the net force load using F = P*A. The converging and diverging sections are linear, whereas the inlet curve requires an integrated formula using the torus shape. More details on this derivation are below.

**Deriving Component Forces on Torus**

Point on Torus: $\vec{r}(\phi) = \langle R + r\cos\phi, r\sin\phi \rangle$  $[115\geq\phi\geq180]$ 

$r$ is the fillet radius. $R$ is the distance between the center of the torus and the center of the minor radius, $r$. $\phi$ is the angle along the arc.

Radius of Circle at $\phi$: $2\pi(R + r\cos\phi)$

Arc Length Element: $rd\phi$

Surface Patch Area: $dA = 2\pi(R + r\cos\phi)rd\phi$

Normal Direction: $\vec{n}(\phi) = \langle \cos\phi, \sin\phi \rangle$

**Force Vector**: $d\bold{F} = pdA\vec{n} = \langle 2\pi{}pr(R + r\cos\phi)\cos\phi d\phi, 2\pi{}pr(R + r\cos\phi)\sin\phi d\phi \rangle$ 

Integrating from $\phi_0 = 115$ to $\phi_1 = 180$ leads to the following integrals for the X and Y components.

**X-Component**
$$F_x = \int_{\phi_0}^{\phi_1} 2\pi{}pr(R + r\cos\phi)\cos\phi d\phi $$

**Y-Component**
$$F_y = \int_{\phi_0}^{\phi_1} 2\pi{}pr(R + r\cos\phi)\sin\phi d\phi $$

Using Wolfram-Alpha to solve the integrals...

$$F_x = 2\pi{}pr(R(\sin\phi_1 - \sin\phi_0) + \frac{1}{4}r(2\phi_1-2\phi_0-sin(2\phi_0) + sin(2\phi_1)))$$
$$F_y = 2\pi{}pr\frac{1}{2}(\cos\phi_0 - \cos\phi_1)(2R + r(\cos\phi_0 + \cos\phi_1)) $$

In [4]:
# Finding the pressures at each section
avg_ps_inlet = []
for i in range(inlet_n-1):
    avg_ps_inlet = np.append(avg_ps_inlet, (ps_inlet[i+1] + ps_inlet[i])/2)
avg_ps_throat = ps_throat
avg_ps_inlet_curve = [np.average(ps_inlet_curve)]
avg_ps_outlet = []
for i in range(outlet_n-1):
    avg_ps_outlet = np.append(avg_ps_outlet, (ps_outlet[i+1] + ps_outlet[i])/2)
avg_ps = np.concatenate((avg_ps_inlet, avg_ps_inlet_curve, avg_ps_throat, avg_ps_outlet))

# Finding force load at each section
Fs_inlet = []
for i in range(inlet_n-1):
    Fs_inlet = np.append(Fs_inlet, avg_ps_inlet[i]*SA_inlet[i])
Fs_throat = [avg_ps_throat[0]*SA_throat[0]]
Fs_outlet = []
for i in range(outlet_n-1):
    Fs_outlet = np.append(Fs_outlet, avg_ps_outlet[i]*SA_outlet[i])
Fs = np.concatenate((Fs_inlet, Fs_throat, Fs_outlet))

# Angles for component forces
phi_0 = np.pi
phi_1 = 115*np.pi/180

# Finding the net vertical force load
F_y_inlet = -Fs_inlet*np.sin(conv_angle*np.pi/180)
F_y_inlet_curve = 2*np.pi*avg_ps_inlet_curve[0]*torus_r/2*(np.cos(phi_0) - np.cos(phi_1))*(2*torus_R + torus_r*(np.cos(phi_0) + np.cos(phi_1)))
F_y_outlet = Fs_outlet*np.sin(div_angle*np.pi/180)
net_F_y = np.sum(F_y_inlet) + F_y_inlet_curve + np.sum(F_y_outlet)

# Finding the net horizontal force load
F_x_inlet = Fs_inlet*np.cos(conv_angle*np.pi/180)
F_x_inlet_curve = 2*np.pi*avg_ps_inlet_curve[0]*torus_r*(torus_R*(np.sin(phi_1) - np.sin(phi_0)) + 1/4*torus_r*(2*phi_1 - 2*phi_0 - np.sin(2*phi_0) + np.sin(2*phi_1)))
F_x_throat = Fs_throat
F_x_outlet = Fs_outlet*np.cos(div_angle*np.pi/180)
net_F_x = np.sum(F_x_inlet) + np.sum(F_x_outlet) + F_x_throat[0] + F_x_inlet_curve

The nozzle washer is the piece that will be holding up the nozzle while it is under load. The area of the nozzle washer that contacts the nozzle will be used to find the stress. The fillet will be simplified to a square corner.

This would be an underestimate because we are not considering any moment forces from the pressure. We are assuming that all forces are acting directly above/lateral to the surfaces of the nozzle washer.

In [7]:
# Finding surface area of nozzle washer fillet
washer_fillet_r = 0.00254
washer_R = 0.028575 + washer_fillet_r
SA_washer_curve = 4*np.pi**2*washer_fillet_r*washer_R*90/360

# Finding surface area of X/Y-directions
SA_washer_x = 2*np.pi*0.05715/2*0.0127
SA_washer_y = np.pi*(0.06985/2)**2 - np.pi*(0.05715/2)**2
# Calculating stresses in X/Y-directions
sigma_x = net_F_x/SA_washer_x
sigma_y = net_F_y/SA_washer_y

# Calculating resultant stress
sigma = np.sqrt(sigma_x**2 + sigma_y**2)
print("The resultant stress in MPa is", round(sigma*1e-6, 5))

The resultant stress in MPa is 10.41317
